In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-34wre2gv
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-34wre2gv
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [ ]:
%%cuda --name JP.cu

#include "/content/drive/MyDrive/graphcoloring/graph/coloring.h"
#include "/content/drive/MyDrive/graphcoloring/graph/graph.h"
#include "/content/drive/MyDrive/graphcoloring/graph/graph_d.h"
#include "/content/drive/MyDrive/graphcoloring/utils/common.h"


#define THREADxBLOCK 128

using namespace std;

__global__ void colorer(Coloring * col, GraphStruct *str){
	int n = str->nodeSize;

	for (int i = threadIdx.x+blockIdx.x*blockDim.x; i < n; i += blockDim.x*gridDim.x) {
		bool flag = true; // vera sse il nodo ha peso locale massimo

		// ignora i nodi già colorati
		if ((col->coloring[i] != -1)) continue;

		int iWeight = str->weights[i];

		// guarda i pesi del vicinato
		uint offset = str->cumDegs[i];
		uint deg = str->cumDegs[i + 1] - str->cumDegs[i];

		for (uint j = 0; j < deg; j++) {
			uint neighID = str->neighs[offset + j];
			// ignora i vicini già colorati (e te stesso)
			int jColor = col->coloring[neighID];
			if (((jColor != -1) && (jColor != col->numOfColors)) || (i == neighID)) continue;
			int jWeight = str->weights[neighID];
			if (iWeight <= jWeight) flag = false;
		}
		// colora solo se sei il nodo di peso massimo
		if (flag) col->coloring[i] = col->numOfColors;
  	}
}

void FYshuffle(int * weights, uint n){
    for(int i = 0; i < n; i++){
        int swapIdx = (rand() % (n - i)) + i;
        int tmp = weights[i];
				weights[i] = weights[swapIdx];
				weights[swapIdx] = tmp;
    }
}

Coloring* graphColoring(GraphStruct *str){
	int n = str->nodeSize;
	Coloring* col;
	CHECK(cudaMallocManaged(&col, sizeof(Coloring)));
	col->uncoloredNodes = true;

    CHECK(cudaMallocManaged(&(col->coloring), n * sizeof(int)));
	memset(col->coloring, -1 ,n * sizeof(int));
	// allocate space on the GPU for the random states

	dim3 threads ( THREADxBLOCK);
	dim3 blocks ((str->nodeSize + threads.x - 1) / threads.x, 1, 1 );

	for (int i = 0; i < n; i++){
        str->weights[i] = i;
    }
    FYshuffle(str->weights, n);

	bool flag = false;
	for(int c = 0; c < n; c++){
		col->numOfColors = c;
		colorer<<<blocks, threads>>>(col, str);
		cudaDeviceSynchronize();

		for(int i=0; i<n; i++){
				if(col->coloring[i] == -1){
						flag = true;
				}
		}

		if(!flag) break;
		else flag = false;

	}

    return col;
}


'File written in /content/src/JP.cu'

In [ ]:
%%cuda --name test_JP.cu

#include "/content/drive/MyDrive/graphcoloring/graph/coloring.h"

int main(void) {
	unsigned int n = 10000;		 // number of nodes for random graphs
	float prob = 0.5;				    // density (percentage) for random graphs
	std::default_random_engine eng{0};  // fixed seed

	srand(time(0));
  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);

	// new graph with n nodes
	Graph graph(n,1);

	// generate a random graph
	graph.randGraph(prob,eng);

	// get the graph struct
	GraphStruct *str = graph.getStruct();

  cudaEventRecord(start);

	//print_d<<<1,1>>>(str, true);

	Coloring* col = graphColoring(str);
	cudaDeviceSynchronize();

	cudaEventRecord(stop);
  cudaEventSynchronize(stop);

	//Stampo in millisecondi quanto tempo ci ha messo a colorare il grafo.
  float milliseconds = 0;
  cudaEventElapsedTime(&milliseconds, start, stop);
  printf("%f ms\n", milliseconds);

	//printColoring(col, str, 1);

	for(int i = 0; i < n; i++){
			printf("%d ", col->coloring[i]);
	}

	printf("\nColore massimo: %d", col->numOfColors);

	return EXIT_SUCCESS;
}

'File written in /content/src/test_JP.cu'

In [ ]:
!nvcc -dc /content/src/test_JP.cu /content/src/JP.cu /content/drive/MyDrive/graphcoloring/graph/graph.cpp /content/drive/MyDrive/graphcoloring/graph/graph_d.cu
!nvcc test_JP.o JP.o graph.o graph_d.o -o test_JP
!./test_JP

9256.614258 ms
5654 200 4428 3524 2519 4400 4449 3450 5290 2114 3628 3636 3467 1206 2927 2961 593 2295 3398 2177 5728 2434 4234 5116 2595 3560 4439 4705 1251 3081 294 3551 3541 2602 3573 3144 2073 5703 3252 4211 2002 1312 686 4962 3184 4510 1675 4264 2933 245 2280 3596 2593 1780 3642 2743 1654 88 1969 5430 2690 1148 3692 1711 5378 2334 24 2209 2290 52 1903 1311 2718 411 2058 2288 5653 4620 2630 1083 4353 2617 760 1020 5032 3782 4441 2376 548 2402 5518 2594 697 4250 3248 928 2040 4724 2585 1454 5471 2903 3746 3713 4896 3763 900 1898 1406 3688 194 1408 131 1906 5620 4731 422 2811 4943 4396 5042 3290 502 3835 2116 5386 3211 4467 591 729 1722 509 2030 438 2752 4224 592 1668 2328 3670 1948 3540 2786 1719 5281 519 1068 5657 981 1717 5441 5275 867 624 5240 3704 1165 2226 4363 81 399 2416 4232 615 2666 4882 2297 3011 5172 938 3169 2240 5783 4315 960 3430 2949 405 2574 1859 1545 3469 2851 4471 2697 5072 1410 5160 4654 2849 3625 5060 2285 4735 2507 3445 2340 5563 4754 4423 3058 4773 2203 4483 53

In [ ]:
!nvprof ./test_JP

==7879== NVPROF is profiling process 7879, command: ./test_JP
10929.926758 ms
1528 4368 3356 1105 579 2616 4956 2941 5628 5071 3168 1233 5371 2406 4127 5160 695 2247 5229 3404 1681 2359 442 5718 4491 4911 3825 4172 747 4823 5018 2786 5756 3427 4566 2234 1443 2386 1500 4830 5278 2248 4740 3606 580 5195 3476 664 3078 69 720 1727 807 200 386 2024 1905 4545 693 4918 5276 423 783 2095 1809 115 5191 757 3387 2712 938 4105 4541 2825 3319 5354 3515 4853 925 120 4125 5439 4509 3383 4502 3600 4800 640 4054 5377 2656 5568 1661 1820 1205 1343 1368 3173 3444 1416 1926 1995 1517 3906 4720 2012 1919 1321 5511 2108 2377 2967 1290 270 3070 3702 4015 2408 3852 5761 5693 5189 3800 848 2378 1220 2009 1787 3220 5458 4382 3329 2532 5608 2564 1131 2065 4619 3582 2703 2055 1376 5488 4571 1644 3114 1314 494 4357 3089 2748 5647 516 2676 2546 1672 4209 2261 723 4074 593 5561 11 5450 2796 2763 3786 5578 5722 4730 606 919 4725 5334 674 1170 2689 3548 1474 5376 2520 2578 4039 5131 916 3313 652 2331 5535 4839 4976 1